#네이버 지식인 Crawling
 - 네이버 지식인에서 여자친구 선물을 검색한 후 데이터 가져오기 

In [ ]:
!apt-get install -y fonts-nanum > /dev/null
!fc-cache -fv > /dev/null
!rm -rf ~/.cache/matplotlib
# Colab 런타임 다시 시작

In [ ]:
# selenium설치
!apt-get update > /dev/null 2>&1
!pip install selenium > /dev/null 2>&1
!apt install chromium-chromedriver > /dev/null 2>&1

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd
from tqdm.notebook import tqdm
from urllib.parse import quote
from bs4 import BeautifulSoup

options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('chromedriver', options=options)
# chromedriver = 'chromedriver.exe'
# driver = webdriver.Chrome(chromedriver)

In [ ]:
base_url = 'https://kin.naver.com/search/list.naver?query='
url = base_url+quote('여친선물')
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
lis = soup.select("ul.basic1 > li")
len(lis)

10

In [ ]:
li = lis[0]
sub_url = li.select_one('a._searchListTitleAnchor')['href']
sub_url

'https://kin.naver.com/qna/detail.naver?d1id=8&dirId=80101&docId=395035694&qb=7Jes7Lmc7ISg66y8&enc=utf8&section=kin&rank=1&search_sort=0&spq=0'

In [ ]:
driver.get(sub_url)
time.sleep(2)

In [ ]:
count=int(driver.find_element_by_css_selector('._answerCount.num').text)
answer_page=(count-1)//5
print(count,answer_page)

8 1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  """Entry point for launching an IPython kernel.


In [ ]:
for i in range(answer_page):
  driver.find_element_by_id("nextPageButton").click()
  time.sleep(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  


In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
divs = soup.select("div.answer-content__item._contentWrap._answer")
len(divs)

8

In [ ]:
div=divs[0]
ps=div.select('p.se-text-paragraph')
for p in ps:
  print(p.get_text())

In [ ]:
divs[6].select('p.se-texxt-paragraph')

In [ ]:
driver.back()
time.sleep(2)

In [ ]:
answer_list = []

for page in tqdm(range(1,11)):
    url = f'{base_url}{quote("여친선물")}&page={page}'
    driver.get(url)
    time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    lis = soup.select('ul.basic1 > li')

    for li in lis:
      sub_url = li.select_one('a._searchListTitleAnchor')['href']
      driver.get(sub_url)
      time.sleep(2)

      count=int(driver.find_element_by_css_selector('._answerCount.num').text)
      answer_page=(count-1)//5
      print(count, end=', ')

      for i in range(answer_page):
        driver.find_element_by_id("nextPageButton").click()
        time.sleep(2)
      
      soup = BeautifulSoup(driver.page_source, 'html.parser')
      divs = soup.select("div.answer-content__item._contentWrap._answer")
      print(len(divs))

      for div in divs:
        ps = div.select('p.se-text-paragraph')
        paragraph =''
        for p in ps:
          text = p.get_text()
          if text == ' ' or text == '\u2200b':
            continue
          paragraph += text + '\n'
        answer_list.append(paragraph)

      driver.back()
      time.sleep(2)

In [ ]:
answer_list[100]

In [ ]:
with open('new_gift.txt', 'w', encoding='utf-8') as fp:
  for answer in answer_list:
    fp.write(answer+'\n')